In [1]:
import pandas as pd
import numpy as np

In [2]:
p = pd.read_csv('./all_pitches.csv')

In [3]:
#need to pull rhh
ab = pd.read_csv('./all_atbats.csv')

In [4]:
filt = p['zone'] != '-1'
p = p.loc[filt]

In [5]:
p['zone'].value_counts()

12a    311062
15a    288733
13a    279727
4a     239050
7a     229461
8a     210612
3a     206808
5a     204035
6a     187835
1a     134066
15b    127702
0a     117847
16b    111816
13b    109396
12b    109055
2a     104105
10b     75499
16a     67266
10a     64783
14b     56338
9b      39108
14a     38355
11b     24377
9a       8575
Name: zone, dtype: int64

In [10]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,zone
0,0.416,0.364,C,S,FF,2015000001,0,0,0,1,0.0,0.0,0.0,0,strike,2a
1,-0.191,-0.252,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike,4a
2,-0.518,0.685,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul,0a
3,-0.641,-1.378,B,B,FF,2015000001,0,2,0,4,0.0,0.0,0.0,0,ball,15b
4,-1.821,-0.516,B,B,CU,2015000001,1,2,0,5,0.0,0.0,0.0,0,ball,12b


In [8]:
filt = ab['stand'] == 'R'
rhh = ab.loc[filt]

In [11]:
#filtering pitches so we have only rhh batters
filt = p['ab_id'].isin(rhh['ab_id'])
p = p.loc[filt]

## Comparing Old Data to New Data to see if our Zone distribution makes sense

In [19]:
# Pulling from swing data from GitHub
import requests, json, io
def pull_github_csv(user='mbendy', pao='ce9aec01691d2a8efde6a73dc3a6a5e337960199'):
    github_session = requests.Session()
    github_session.auth = (user, pao)

    # providing raw url to download csv from github
    url_swing = 'https://raw.githubusercontent.com/everwitt7/Pitch-Data/master/all_pitchers_rhh_swing_data.csv'
    url_take = 'https://raw.githubusercontent.com/everwitt7/Pitch-Data/master/all_pitchers_rhh_take_data.csv'
    
    download = github_session.get(url_swing).content
    df_swing = pd.read_csv(io.StringIO(download.decode('utf-8')), error_bad_lines=False)
    
    download = github_session.get(url_take).content
    df_take = pd.read_csv(io.StringIO(download.decode('utf-8')), error_bad_lines=False)
    
    df = df_swing.append(df_take, ignore_index=False)

    # pulling only necessary data  
    pitcher_actions = ["FF", "SL", "FT", "CH", "CU", "FC", "SI"]
    df = df[df['pitch_type'].isin(pitcher_actions)]

    return df

old_data = pull_github_csv()

In [20]:
old_data['zones'].value_counts(normalize=True)

13a    0.104323
15a    0.084263
8a     0.075859
4a     0.075046
5a     0.072232
7a     0.069566
12a    0.069282
3a     0.052907
6a     0.044822
13b    0.042761
16b    0.042073
1a     0.040901
15b    0.035156
2a     0.033747
0a     0.031728
16a    0.024925
10b    0.021490
12b    0.020445
10a    0.018259
14b    0.013108
11b    0.008926
14a    0.008818
9b     0.007576
9a     0.001786
Name: zones, dtype: float64

In [21]:
p['zone'].value_counts(normalize=True)

13a    0.104544
15a    0.084211
8a     0.075513
4a     0.073526
5a     0.071406
12a    0.068383
7a     0.068323
3a     0.052189
6a     0.044446
16b    0.042946
13b    0.042334
1a     0.041136
15b    0.036388
2a     0.033961
0a     0.032117
16a    0.026069
10b    0.022214
12b    0.020632
10a    0.018996
14b    0.013630
11b    0.008989
14a    0.008509
9b     0.007412
9a     0.002124
Name: zone, dtype: float64

In [22]:
print(old_data.shape)
print(p.shape)

(1575773, 12)
(1958772, 16)


In [29]:
#want to group by pitch and zone, then I want the normalized value counts of 'result'
pz_group = p.groupby(['pitch_type', 'zone'])
trans = pz_group['res'].value_counts(normalize=True)

In [31]:
#the shape should be num_pitches * num_zones * num_res ~ 6 * 22 * 5 = 660
trans.shape

(667,)

In [32]:
trans

pitch_type  zone  res   
CH          0a    strike    0.375752
                  foul      0.266032
                  ball      0.159319
                  out       0.138778
                  hit       0.060120
                              ...   
SL          9b    ball      0.967495
                  strike    0.016571
                  foul      0.011472
                  out       0.003187
                  hit       0.001275
Name: res, Length: 667, dtype: float64

In [45]:
trans['FF']['0a']['strike']

0.33159192298661655

In [48]:
t = p.groupby(['pitch_type', 'zone', 'on_1b'])
tt = t['res'].value_counts(normalize=True)

In [49]:
tt

pitch_type  zone  on_1b  res   
CH          0a    0.0    strike    0.365123
                         foul      0.248865
                         ball      0.166213
                         out       0.138056
                         hit       0.081744
                                     ...   
SL          9b    0.0    out       0.002273
                  1.0    ball      0.961440
                         strike    0.020566
                         foul      0.012853
                         out       0.005141
Name: res, Length: 1317, dtype: float64